In [2]:
import pandas as pd
import numpy as np
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k
from lightfm import LightFM

from hyperopt import fmin, hp, tpe, Trials
from scipy.io import mmwrite

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [3]:
pd.set_option('display.max_columns', None)

In [76]:
transaction = pd.read_json('./transaction_top.json')
cloth_meta = pd.read_json('./final_top.json')
user_meta = pd.read_json('./user_top.json')

In [77]:
transaction = transaction[transaction.shopCnt > 1]
transaction.insert(0, 'largecategory', 1, True)
transaction = transaction.drop('shopCnt', axis=1)
transaction.shape

(281090, 3)

In [78]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]
transaction.index = range(len(transaction))

In [79]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta.index = range(len(user_meta))

In [80]:
unique_cloth_id = list(transaction['newClothId'].unique())
cloth_meta = cloth_meta.loc[cloth_meta.newClothId.isin(unique_cloth_id), :]
cloth_meta.index = range(len(cloth_meta))

In [81]:
transaction

,largecategory,userId,newClothId
0,1,858064,2149254153363
1,1,858065,1841247138594
2,1,858067,2038817208900
3,1,858070,1876366243372
4,1,858071,2222074176235
...,...,...,...
281085,1,1416946,1558847204939
281086,1,1416946,1666085258669
281087,1,1416946,6657950174519
281088,1,1416948,1912952134031


In [82]:
user_meta

,userId,userName,largecategory,userMale,userFemale,userHeight,userWeight,size,bright,color,thickness,weightness,touch,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,clothCnt
0,858064,!!!!!!!!,1,1,0,183,90,1.250000,1.000000,1.0,2.750000,0,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.666667,0.0,0.0,0.666667,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.333333,0,0,0,0,0,0,0,0,0,0,3,1,4
1,858065,!!!!!!!!!!,1,0,1,164,57,1.600000,1.800000,2.0,2.200000,0,0.000000,0.500000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.750000,0.000000,0.250000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0,0,1,0,1,1,0,0,0,0,2,0,5
2,858067,!!!!!!!!@,1,0,1,172,59,1.750000,2.000000,2.0,1.500000,0,0.000000,0.250000,0.000000,0.0,0.500000,0.250000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.500000,0.0,0.000000,0.250000,0.0,0.000000,0.0,0.0,0.000000,0.250000,0,3,0,0,0,1,0,0,0,0,0,0,4
3,858070,!!!::!!,1,1,0,167,70,1.000000,2.000000,1.0,3.000000,0,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0,0,0,0,0,0,2,0,0,0,0,0,2
4,858071,!!!@!,1,1,0,177,68,2.000000,1.800000,2.0,2.000000,0,0.800000,0.400000,0.000000,0.0,0.000000,0.000000,0.400000,0.200000,0.0,0.0,0.000000,0.000000,0.800000,0.0,0.0,0.000000,0.000000,0.0,0.200000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0,0,0,1,0,0,2,0,0,0,0,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131310,1416935,ｎｏ,1,1,0,183,77,1.866667,2.000000,1.6,2.133333,0,0.466667,0.214286,0.000000,0.0,0.071429,0.214286,0.285714,0.214286,0.0,0.0,0.071429,0.214286,0.142857,0.0,0.0,0.071429,0.000000,0.0,0.071429,0.285714,0.0,0.071429,0.0,0.0,0.071429,0.000000,0,0,0,0,0,1,0,3,0,6,5,0,15
131311,1416936,ｎｏｄｉ,1,1,0,179,78,1.333333,1.666667,2.0,1.333333,0,1.333333,0.000000,0.333333,0.0,0.000000,0.000000,0.666667,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.333333,0.000000,0.0,0.666667,0.0,0.0,0.000000,0.000000,0,2,0,0,0,0,0,0,0,0,1,0,3
131312,1416938,｡･ω･｡,1,0,1,171,60,1.600000,2.200000,2.2,2.200000,0,0.200000,0.000000,0.000000,0.0,0.400000,0.400000,0.200000,0.000000,0.0,0.0,0.000000,0.000000,0.200000,0.0,0.0,0.000000,0.200000,0.0,0.000000,0.600000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,5,5
131313,1416946,￦onyee,1,1,0,175,62,2.000000,2.000000,2.0,2.166667,0,0.333333,0.166667,0.000000,0.0,0.166667,0.166667,0.166667,0.333333,0.0,0.0,0.166667,0.000000,0.416667,0.0,0.0,0.000000,0.166667,0.0,0.000000,0.166667,0.0,0.000000,0.0,0.0,0.000000,0.083333,0,0,0,0,2,0,0,0,0,2,4,4,12


In [83]:
cloth_meta

,newClothId,clothId,largeCategory,largeCategoryName,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers,smallCategoryName,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,colorName,clothName,brand,clothImg,clothPrice,date,clothHashtags,clothMale,clothFemale,clothRate,clothReviewCnt,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter,goodsSize,reviewNoun,userHeight,userWeight,userMale,userFemale,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,1440513131686,1440513,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,white,유니섹스 레귤러핏 유스 로고 티-화이트,필루미네이트,//image.msscdn.net/images/goods_img/20200512/1...,18000,20200512,"[그래픽반팔, 그래픽티, 레귤러핏반팔, 반팔, 반팔티, 티셔츠]",1,1,4.4,10,0,0,0,0,0,0,0,0,0,L,"[{'text': '사이즈', 'value': 3}, {'text': '더', 'v...",169.750000,59,0.500000,0.500000,0,0,0,1,1,0,0,1,0,1,0,0
1,1930909131765,1930909,1,상의,0,0,0,0,0,0,1,0,0,긴팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,white,에센셜 수피마 롱 슬리브 화이트,엘무드,//image.msscdn.net/images/goods_img/20210430/1...,38700,20210430,"[롱슬리브, 긴팔티셔츠, 면티, 루즈핏, 무지, 베이식, 오버핏]",1,0,4.9,158,4,1,5,4,3,1,1,1,1,44,"[{'text': '핏', 'value': 15}, {'text': '구매', 'v...",163.600000,56,1.000000,1.000000,0,2,0,0,0,1,0,0,1,0,0,1
2,1930909131764,1930909,1,상의,0,0,0,0,0,0,1,0,0,긴팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,white,에센셜 수피마 롱 슬리브 화이트,엘무드,//image.msscdn.net/images/goods_img/20210430/1...,38700,20210430,"[롱슬리브, 긴팔티셔츠, 면티, 루즈핏, 무지, 베이식, 오버핏]",1,0,4.9,158,4,1,5,4,3,1,1,1,1,46,"[{'text': '핏', 'value': 15}, {'text': '구매', 'v...",169.666667,64,0.666667,0.333333,0,0,0,0,0,0,0,0,0,0,0,3
3,1930909131766,1930909,1,상의,0,0,0,0,0,0,1,0,0,긴팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,white,에센셜 수피마 롱 슬리브 화이트,엘무드,//image.msscdn.net/images/goods_img/20210430/1...,38700,20210430,"[롱슬리브, 긴팔티셔츠, 면티, 루즈핏, 무지, 베이식, 오버핏]",1,0,4.9,158,4,1,5,4,3,1,1,1,1,48,"[{'text': '핏', 'value': 15}, {'text': '구매', 'v...",166.500000,59,0.500000,0.500000,0,0,0,0,0,0,0,0,1,1,0,0
4,1931411131771,1931411,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,white,러디 레트로 페이퍼 반팔 티셔츠 (WHITE),트릴리온,//image.msscdn.net/images/goods_img/20210430/1...,19900,20210430,"[그래픽, 오버핏, 루즈핏, 크루넥, 숏슬리브, 반팔셔츠, 레이어드]",1,0,4.7,38,4,3,3,5,3,1,1,1,1,XL,"[{'text': '프린팅', 'value': 4}, {'text': '디자인', ...",175.000000,57,1.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59814,1281638267701,1281638,1,상의,0,0,0,1,0,0,0,0,0,후드 티셔츠,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,others,TIE DYE 후드티 Olive Green,크리틱,//image.msscdn.net/images/goods_img/20200128/1...,69300,20200128,"[물나염, 타이다이, 후드, 패밀리세일]",1,0,4.2,41,0,0,0,0,0,0,0,0,0,XL,"[{'text': '생각', 'value': 3}, {'text': '버핏', 'v...",161.666667,56,1.000000,1.000000,0,0,0,0,0,0,0,0,0,3,0,0
59815,1622670267773,1622670,1,상의,0,1,0,0,0,0,0,0,0,셔츠/블라우스,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,others,CB 아콘 피그먼트 셔츠자켓 (브라운),카고브로스,//image.msscdn.net/images/goods_img/20200924/1...,52000,20200924,"[셔츠, 남자셔츠, 긴팔셔츠, 오버사이즈셔츠, 오버핏셔츠, 피그먼트셔츠]",1,0,4.7,51,0,0,0,0,0,0,0,0,0,FREE,"[{'text': '방', 'value': 2}, {'text': '봄', 'val...",174.750000,61,1.000000,1.000000,0,0,0,0,0,0,0,0,0,0,4,0
59816,1556851267795,1556851,1,상의,0,0,0,0,0,1,0,0,0,니트/스웨터,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,others,MULTI PATTERN BLOCK KNIT (GREEN MULTI),컨츄리파우더,//image.msscdn.net/images/goods_img/20200820/1...,25000,20200820,"[니트, 루즈핏, 스웨터, 오버핏, 자가드]",1,0,4.9,10,5,3,4,5,3,0,0,1,0,M,"[{'text': '왜', 'value': 2}, {'text': '이', 'val...",170.000000,70,1.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,1
59817,1651849267836,1651849,1,상

In [36]:
transaction.to_json('transaction_top.json', orient='records', force_ascii=False)
user_meta.to_json('user_top.json', orient='records', force_ascii=False)
cloth_meta.to_json('final_top.json', orient='records', force_ascii=False)

In [97]:
transaction = pd.read_json('./transaction_outer.json')
cloth_meta = pd.read_json('./final_outer.json')
user_meta = pd.read_json('./user_outer.json')

In [98]:
transaction = transaction[transaction.shopCnt > 1]
transaction = transaction.drop(['largecategory', 'shopCnt'], axis=1)
transaction.insert(0, 'largecategory', 2, True)
transaction.shape

(171253, 3)

In [99]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]
transaction.index = range(len(transaction))

In [100]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta.index = range(len(user_meta))

In [101]:
unique_cloth_id = list(transaction['newClothId'].unique())
cloth_meta = cloth_meta.loc[cloth_meta.newClothId.isin(unique_cloth_id), :]
cloth_meta.index = range(len(cloth_meta))

In [102]:
transaction

,largecategory,userId,newClothId
0,2,488503,117338595070
1,2,488504,168749091775
2,2,488506,128057696626
3,2,488507,163980592715
4,2,488509,186956083291
...,...,...,...
163851,2,858057,174443292358
163852,2,858058,1404195124429
163853,2,858060,208083683393
163854,2,858060,208084081907


In [103]:
user_meta

,userId,userName,largecategory,userMale,userFemale,userHeight,userWeight,size,bright,color,thickness,weightness,touch,smallCategoryHoodie,smallCategoryBlouson,smallCategoryRiders,smallCategoryMustang,smallCategoryCardigan,smallCategoryFleece,smallCategoryCoat,smallCategoryPaddedcoat,smallCategoryVest,smallCategoryJacket,smallCategoryOthers,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,clothCnt
0,488503,! . !,2,1,0,164,50,2.0,2.000000,2.000000,0.000000,2.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,1,2
1,488504,!!!!!!!!!!,2,0,1,164,57,2.0,2.000000,3.000000,0.000000,2.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,2,0,2
2,488506,!!!!!!!!@,2,0,1,172,62,2.0,2.000000,2.000000,3.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,2,0,0,0,0,0,0,0,0,0,0,2
3,488507,!!!@!,2,1,0,177,68,2.0,2.000000,1.666667,2.000000,0.000000,0.0,0.0,0.000000,0.000000,0.666667,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,2,3
4,488509,!!??!!?,2,0,1,157,43,1.0,2.000000,2.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93245,858056,ＸΛｎＴΛ,2,1,0,177,70,2.2,2.000000,1.400000,1.600000,0.600000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.4,0.0,0.600000,0.0,0.0,0.0,0.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.4,0.0,0.0,0.2,0,0,0,0,0,0,0,0,0,2,2,1,5
93246,858057,ｎｏ,2,1,0,183,76,2.0,2.333333,1.000000,2.333333,1.333333,0.0,0.0,0.333333,0.666667,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,1,0,0,0,0,0,0,0,1,3
93247,858058,ｎｏｄｉ,2,1,0,179,78,2.0,3.000000,2.000000,3.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0,2,0,0,0,0,0,0,0,0,0,0,2
93248,858060,･ω･,2,1,0,169,57,1.0,1.000000,1.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.5,0.0,0.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4,0,0,0,0,0,0,0,0,0,0,0,4


In [104]:
cloth_meta

,newClothId,clothId,largeCategory,largeCategoryName,smallCategoryHoodie,smallCategoryBlouson,smallCategoryRiders,smallCategoryMustang,smallCategoryCardigan,smallCategoryFleece,smallCategoryCoat,smallCategoryPaddedcoat,smallCategoryVest,smallCategoryJacket,smallCategoryOthers,smallCategoryName,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,colorName,clothName,brand,clothImg,clothPrice,date,clothHashtags,clothMale,clothFemale,clothRate,clothReviewCnt,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter,goodsSize,reviewNoun,userHeight,userWeight,userMale,userFemale,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,234005278848,2340052,2,아우터,0,0,0,0,1,0,0,0,0,0,0,카디건,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,pink,[NF] 믹스 버튼 가디건 (DEEP PINK)_F22QB401,네스티팬시클럽,//image.msscdn.net/images/goods_img/20220204/2...,38700,20220204,"[크롭가디건, 가디건, 데일리가디건, 여자가디건, 꾸안꾸, 봄아우터]",0,1,4.9,16,2,3,3,5,3,0,0,0,0,OS,"[{'text': '약간', 'value': 4}, {'text': '분', 'va...",183.5,72,1.0,1.0,1,0,0,0,0,0,0,0,0,0,0,1
1,208155079211,2081550,2,아우터,0,0,0,0,0,0,0,1,0,0,0,숏패딩/숏헤비 아우터,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,orange,NJ1DM62C 남성 1996 에코 눕시 자켓,노스페이스,//image.msscdn.net/images/goods_img/20210823/2...,299000,20210823,"[헤비아우터, 바람막이, 다운점퍼, 아우터, 구스다운, 눕시컬렉션, 숏패딩, 에코컬렉션]",1,0,4.8,4508,0,0,0,0,0,0,0,0,0,ORANGE/095(M),"[{'text': '생각', 'value': 7}, {'text': '좀', 'va...",174.5,71,1.0,0.0,0,1,1,0,0,0,0,0,0,0,0,0
2,208155079212,2081550,2,아우터,0,0,0,0,0,0,0,1,0,0,0,숏패딩/숏헤비 아우터,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,orange,NJ1DM62C 남성 1996 에코 눕시 자켓,노스페이스,//image.msscdn.net/images/goods_img/20210823/2...,299000,20210823,"[헤비아우터, 바람막이, 다운점퍼, 아우터, 구스다운, 눕시컬렉션, 숏패딩, 에코컬렉션]",1,0,4.8,4508,0,0,0,0,0,0,0,0,0,ORANGE/100(L),"[{'text': '생각', 'value': 7}, {'text': '좀', 'va...",168.0,79,1.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0
3,208155079213,2081550,2,아우터,0,0,0,0,0,0,0,1,0,0,0,숏패딩/숏헤비 아우터,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,orange,NJ1DM62C 남성 1996 에코 눕시 자켓,노스페이스,//image.msscdn.net/images/goods_img/20210823/2...,299000,20210823,"[헤비아우터, 바람막이, 다운점퍼, 아우터, 구스다운, 눕시컬렉션, 숏패딩, 에코컬렉션]",1,0,4.8,4508,0,0,0,0,0,0,0,0,0,ORANGE/105(XL),"[{'text': '생각', 'value': 7}, {'text': '좀', 'va...",183.0,95,1.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0
4,208155079214,2081550,2,아우터,0,0,0,0,0,0,0,1,0,0,0,숏패딩/숏헤비 아우터,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,orange,NJ1DM62C 남성 1996 에코 눕시 자켓,노스페이스,//image.msscdn.net/images/goods_img/20210823/2...,299000,20210823,"[헤비아우터, 바람막이, 다운점퍼, 아우터, 구스다운, 눕시컬렉션, 숏패딩, 에코컬렉션]",1,0,4.8,4508,0,0,0,0,0,0,0,0,0,ORANGE/110(XXL),"[{'text': '생각', 'value': 7}, {'text': '좀', 'va...",168.0,65,1.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25911,2080918130502,2080918,2,아우터,0,0,0,0,0,0,0,0,0,1,0,나일론/코치 재킷,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,black,윈드브레이커 자켓 (블랙),쿠어,//image.msscdn.net/images/goods_img/20210823/2...,152100,20210823,"[쿠어자켓, 자켓, 윈드브레이커, 바람막이, 간절기아우터, 21F/W아우터]",1,1,4.9,279,0,0,0,0,0,0,0,0,0,M,"[{'text': '쿠어', 'value': 35}, {'text': '지퍼', '...",177.0,70,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,2
25912,2080918130504,2080918,2,아우터,0,0,0,0,0,0,0,0,0,1,0,나일론/코치 재킷,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,black,윈드브레이커 자켓 (블랙),쿠어,//image.msscdn.net/images/goods_img/20210823/2...,152100,20210823,"[쿠어자켓, 자켓, 윈드브레이커, 바람막이, 간절기아우터, 21F/W아우터]",1,1,4.9,279,0,0,0,0,0,0,0,0,0,S,"[{'text': '쿠어', 'value': 35}, {'text': '지퍼', '...",155.0,43,1.0,1.0,0,0,0,0,0,0,0,0,2,0,0,0
25913,2322135130506,2322135,2,아우터,0,0,0,0,0,0,0,0,0,1,0,스타디움 재킷,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,black,딜런 스카잔 바시티 재킷 블랙,엘무드,//image.msscdn.net/images/goods_img/20220124/2...,169200,20220124,"[22S/S2차발매, 22S/S컬렉션, 바시티, 스카잔, 세미오버, 크롭, 윈드브레이커]",1,0,4.

In [105]:
transaction.to_json('transaction_outer.json', orient='records', force_ascii=False)
user_meta.to_json('user_outer.json', orient='records', force_ascii=False)
cloth_meta.to_json('final_outer.json', orient='records', force_ascii=False)

In [112]:
transaction = pd.read_json('./transaction_pants.json')
cloth_meta = pd.read_json('./final_pants.json')
user_meta = pd.read_json('./user_pants.json')

In [114]:
transaction = transaction[transaction.shopCnt > 1]
transaction = transaction.drop(['largecategory', 'shopCnt'], axis=1)
transaction.insert(0, 'largecategory', 3, True)
transaction.shape

(273862, 3)

In [115]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]
transaction.index = range(len(transaction))

In [116]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta.index = range(len(user_meta))

In [117]:
unique_cloth_id = list(transaction['newClothId'].unique())
cloth_meta = cloth_meta.loc[cloth_meta.newClothId.isin(unique_cloth_id), :]
cloth_meta.index = range(len(cloth_meta))

In [118]:
transaction

,largecategory,userId,newClothId
0,3,14135,167945567571
1,3,14141,16161698056
2,3,14146,156805770706
3,3,14148,211205952557
4,3,14148,396185072810
...,...,...,...
253576,3,488483,174703778685
253577,3,488490,180476573410
253578,3,488491,817936052454
253579,3,488502,101577772513


In [119]:
user_meta

,userId,userName,largecategory,userMale,userFemale,userHeight,userWeight,size,bright,color,thickness,weightness,touch,smallCategoryDenimpants,smallCategoryCottonpants,smallCategorySlacks,smallCategoryJoggerpants,smallCategoryShortpants,smallCategoryLeggings,smallCategoryJumpsuit,smallCategoryOthers,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,clothCnt
0,14135,!!!!!!!!@,3,0,1,172,59,1.428571,2.000000,2.000000,2.000000,0.0,0,0.666667,0.000000,0.000000,0.333333,0.000000,0.0,0.0,0.000000,0.000000,0.333333,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.666667,0.0,0.000000,3,0,0,0,0,1,0,3,0,0,0,0,7
1,14141,!!어흥!!,3,1,0,182,77,1.937500,2.000000,2.000000,2.250000,0.0,0,0.407407,0.222222,0.185185,0.185185,0.000000,0.0,0.0,0.000000,0.037037,0.148148,0.222222,0.0,0.0,0.0,0.000000,0.0,0.037037,0.000000,0.0,0.000000,0.185185,0.333333,0.0,0.037037,6,12,3,2,2,0,2,0,0,0,0,5,32
2,14146,!-¡,3,0,1,162,51,1.000000,2.000000,2.000000,2.000000,0.0,0,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.666667,0.333333,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0,0,0,0,0,0,0,0,0,0,3,0,3
3,14148,!0박주현!,3,1,0,180,68,2.000000,2.000000,2.000000,2.000000,0.0,0,0.500000,0.000000,0.000000,0.500000,0.000000,0.0,0.0,0.000000,0.000000,0.500000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.500000,0.0,0.000000,2,2,0,0,0,0,0,0,0,0,0,0,4
4,14158,!@ㅇㄻ,3,1,0,158,57,1.870968,1.870968,2.000000,2.096774,0.0,0,0.500000,0.200000,0.150000,0.150000,0.000000,0.0,0.0,0.000000,0.100000,0.300000,0.100000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.100000,0.200000,0.200000,0.0,0.000000,2,1,0,8,2,3,1,1,9,1,1,2,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115960,488479,＂ㅁㅇㄻㄴㄹㅇ,3,1,0,180,78,1.884615,1.846154,1.846154,2.000000,0.0,0,0.423077,0.153846,0.230769,0.153846,0.000000,0.0,0.0,0.038462,0.000000,0.038462,0.192308,0.0,0.0,0.0,0.115385,0.0,0.115385,0.000000,0.0,0.076923,0.153846,0.192308,0.0,0.115385,7,4,0,4,0,0,0,0,6,1,1,3,26
115961,488483,Ｇｒａｃｅ,3,0,1,168,56,1.000000,2.200000,1.000000,1.400000,0.0,0,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.200000,0.800000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0,0,0,0,0,0,0,4,0,0,0,1,5
115962,488490,ｎｏ,3,1,0,183,77,1.583333,2.000000,1.916667,2.083333,0.0,0,0.181818,0.272727,0.000000,0.272727,0.181818,0.0,0.0,0.090909,0.000000,0.454545,0.090909,0.0,0.0,0.0,0.181818,0.0,0.000000,0.090909,0.0,0.000000,0.090909,0.090909,0.0,0.000000,1,1,1,0,0,0,0,3,0,2,3,1,12
115963,488491,ｎｏｄｉ,3,1,0,179,78,2.000000,1.000000,2.000000,2.000000,0.0,0,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0,2,0,0,0,0,0,0,0,0,0,0,2


In [120]:
cloth_meta

,newClothId,clothId,largeCategory,largeCategoryName,smallCategoryDenimpants,smallCategoryCottonpants,smallCategorySlacks,smallCategoryJoggerpants,smallCategoryShortpants,smallCategoryLeggings,smallCategoryJumpsuit,smallCategoryOthers,smallCategoryName,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,colorName,clothName,brand,clothImg,clothPrice,date,clothHashtags,clothMale,clothFemale,clothRate,clothReviewCnt,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter,goodsSize,reviewNoun,userHeight,userWeight,userMale,userFemale,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,16225354884,1622535,3,바지,1,0,0,0,0,0,0,0,데님 팬츠,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,grey,와이드 데님 팬츠 그레이,모티브스트릿,//image.msscdn.net/images/goods_img/20200924/1...,45000,20200924,"[와이드팬츠, 와이드, 데님, 바지, 팬츠, 와이드데님]",1,0,4.6,156,4,3,5,5,3,1,1,1,1,L,"[{'text': '핏', 'value': 2}, {'text': '워싱', 'va...",183.0,78,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,2
1,14988915210,1498891,3,바지,0,1,0,0,0,0,0,0,코튼 팬츠,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,grey,NeB summer cotton pants (gray),누프에블루,//image.msscdn.net/images/goods_img/20200626/1...,28700,20200626,"[누블, 바지, 코튼팬츠, 와이드팬츠, 캐주얼팬츠, 밴딩팬츠, 스트레이트]",1,0,5.0,2,0,0,0,0,0,0,0,0,0,NONE,"[{'text': '허리', 'value': 2}, {'text': '끈', 'va...",165.0,60,1.0,0.0,0,0,0,1,1,0,0,0,0,0,0,0
2,19409615513,1940961,3,바지,0,1,0,0,0,0,0,0,코튼 팬츠,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,grey,썸머 스트링 팬츠_웜그레이,노드스튜디오,//image.msscdn.net/images/goods_img/20210507/1...,58000,20210507,"[여름바지, 캐주얼팬츠, 밴딩팬츠, 코튼팬츠, 면바지, 셋업팬츠]",1,0,5.0,2,3,2,2,3,2,1,1,1,0,3(L31~33),"[{'text': '예상', 'value': 2}, {'text': '마감', 'v...",174.0,62,1.0,0.0,0,0,0,0,0,0,0,2,0,0,0,0
3,20193975542,2019397,3,바지,0,0,0,0,1,0,0,0,숏 팬츠,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,grey,UNISEX 버뮤다 원턱 우븐밴딩 하프팬츠 그레이(FCB2PT301M),패블러스아카이브,//image.msscdn.net/images/goods_img/20210705/2...,59000,20210705,"[스트릿패션, 반바지, 면바지, 쇼트팬츠, 하프팬츠, 밴딩반바지, 쇼츠]",1,0,4.0,2,0,0,0,0,0,0,0,0,0,그레이/M,"[{'text': '바지', 'value': 4}, {'text': '핏', 'va...",175.0,67,1.0,0.0,0,0,0,0,0,0,0,0,0,1,1,0
4,18466875743,1846687,3,바지,0,1,0,0,0,0,0,0,코튼 팬츠,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,grey,PINTUCK WIDE SWEAT PANTS grey,객,//image.msscdn.net/images/goods_img/20210316/1...,55300,20210316,"[바지, 코튼, 캐주얼팬츠, 코튼팬츠, 와이드팬츠, 밴딩팬츠, 와이드]",1,0,5.0,6,5,3,3,5,3,1,0,1,0,M,"[{'text': '구매', 'value': 2}, {'text': '질도', 'v...",176.0,100,1.0,0.0,0,0,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33525,180323169094,1803231,3,바지,1,0,0,0,0,0,0,0,데님 팬츠,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,jean,OPT TURNING DENIM PANTS ICE,메종미네드,//image.msscdn.net/images/goods_img/20210218/1...,81800,20210218,[[]],1,0,4.8,208,0,0,0,0,0,0,0,0,0,46,"[{'text': '핏', 'value': 7}, {'text': '색감', 'va...",175.0,57,1.0,0.0,0,1,0,0,1,0,0,2,1,0,0,1
33526,182037869103,1820378,3,바지,1,0,0,0,0,0,0,0,데님 팬츠,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,jean,PAUL209 스트레이트 크롭핏 데님진 (연청),폴코어,//image.msscdn.net/images/goods_img/20210302/1...,29400,20210302,"[데님팬츠, 폴코어신상, 스트레이트, 청바지, 데님, 남자청바지, 팬츠]",1,0,4.7,433,0,0,0,0,0,0,0,0,0,50(33~34.5),"[{'text': '가성', 'value': 3}, {'text': '비', 'va...",177.0,75,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0
33527,194119769137,1941197,3,바지,1,0,0,0,0,0,0,0,데님 팬츠,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,jean,Pintuck Stitch Jeans (Light Blue),바이탈싸인,//image.msscdn.net/images/goods_img/20210507/1...,84550,20210507,"[데님, 청바지, 와이드데님, 진, 데님팬츠, 핀턱팬츠]",0,1,4.5,6,0,0,0,0,0,0,0,0,0,M,"[{'text': '막', 'value': 3}, {'text': '허리', 'va...",160.0,52,0.0,1.0,0,0,0,0,0,0,1,0,0,0,0,0
33528,179115969145,1791159,3,바지,1,0,0,0,0,0,0,0,데님 팬츠,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,jean,TAG DAMAGED DENIM PANTS - LIGHT BLUE,브라운브레스,//image.msscdn.net/imag

In [121]:
transaction.to_json('transaction_pants.json', orient='records', force_ascii=False)
user_meta.to_json('user_pants.json', orient='records', force_ascii=False)
cloth_meta.to_json('final_pants.json', orient='records', force_ascii=False)

In [122]:
transaction = pd.read_json('./transaction_onepiece.json')

In [124]:
transaction = transaction.drop(['largecategory', 'shopCnt'], axis=1)
transaction.insert(0, 'largecategory', 4, True)

In [127]:
transaction.to_json('transaction_onepiece.json', orient='records', force_ascii=False)

In [128]:
transaction = pd.read_json('./transaction_skirt.json')

In [129]:
transaction = transaction.drop(['largecategory', 'shopCnt'], axis=1)
transaction.insert(0, 'largecategory', 5, True)

In [130]:
transaction.to_json('transaction_skirt.json', orient='records', force_ascii=False)

In [146]:
cloth_top = pd.read_json('./category_top_cloth_meta.json')
cloth_outer = pd.read_json('./category_outer_cloth_meta.json')
cloth_pants = pd.read_json('./category_pants_cloth_meta.json')
cloth_onepiece = pd.read_json('./category_onepiece_cloth_meta.json')
cloth_skirt = pd.read_json('./category_skirt_cloth_meta.json')

In [147]:
cloth_top = cloth_top.drop('what', axis=1)
cloth_top.insert(0, 'what', 1, True)

In [148]:
cloth_outer = cloth_outer.drop('what', axis=1)
cloth_outer.insert(0, 'what', 2, True)

In [149]:
cloth_pants = cloth_pants.drop('what', axis=1)
cloth_pants.insert(0, 'what', 3, True)

In [150]:
cloth_onepiece = cloth_onepiece.drop('what', axis=1)
cloth_onepiece.insert(0, 'what', 4, True)

In [151]:
cloth_skirt = cloth_skirt.drop('what', axis=1)
cloth_skirt.insert(0, 'what', 5, True)

In [152]:
cloth_top.to_json('cloth_meta_top.json', orient='records', force_ascii=False)
cloth_outer.to_json('cloth_meta_outer.json', orient='records', force_ascii=False)
cloth_pants.to_json('cloth_meta_pants.json', orient='records', force_ascii=False)
cloth_onepiece.to_json('cloth_meta_onepiece.json', orient='records', force_ascii=False)
cloth_skirt.to_json('cloth_meta_skirt.json', orient='records', force_ascii=False)